# Установка и импорт библиотек

In [ ]:
!pip3 install langgraph
!pip3 install -U langchain-groq

In [ ]:
from typing import TypedDict, List, Annotated, Literal
from langgraph.graph.message import add_messages
from pandas.api.types import is_numeric_dtype

from langgraph.graph import StateGraph, START, END
from langchain_groq import ChatGroq
from langgraph.graph import StateGraph
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import SystemMessage
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from pydantic import BaseModel, Field

import pandas as pd
import json

# Инициализация llm

In [ ]:
llm = ChatGroq(
    model='llama-3.3-70b-versatile'
    , temperature=0.1
    , api_key=''
)

# Логика графа

In [ ]:
class UserState(TypedDict):
    # Инициализируется сразу
    messages: Annotated[list, add_messages] # - сообщение, с которой пришел пользователь
    data: pd.DataFrame                      # - датасет

    target: str                             # - переменной по которой будем делать стат тесты
    grouping_variable: str                  # - все переменные которые пойдут в логику исследования
    grouping_values: List[str]              # - разделюящие группирующую переменную на выборки
    satisfied: str                          # - состояние выбронных переменных
    is_normal_target: bool                  # - нормальное ли распределение таргета

    samples: List[str]                      # - группы для стат теста

    target_dtype: str                       # - тип целевой переменной
    grouping_variable_dtype: str            # - тип группирующей переменной

    method_of_var_analisis: str             # - метод дисперсионного анализа**

    artefacts: List[str]                    # - все что будет переданно в llm для вывода сводки

    summary: str                            # - сводка исследования

## Определение переменных исследования

In [ ]:
class TargetExtraction(BaseModel):
    target: str = Field(description='Название зависимой переменной. По которой будет производиться стат тест. Тип может быть любой.')
    grouping_variable: str = Field(description='Группирующая переменная. Если подразумеватеся проверка нескольких групп, рекомендуется использовать одну переменную. Тип может быть любой.')
    grouping_values: List[str] = Field(description='Значения, разделюящие группирующую переменную на выборки (в том числе и какие-то значение числовой переменной, НО передавать строго списком, из str значений). Также тут можем быть СПИСОК с одним значением "ALL", если к примеру, необходимо провести анализ взаимосвязи двх количественных переменных.')

def find_variables(state: UserState) -> UserState:
    data = state['data']
    all_counts = {}
    for col in data.columns:
        counts = data[col].value_counts().head(10).to_dict()
        if len(data[col].unique()) > 10:
            counts['...'] = f'and {len(data[col].unique()) - 10} more'

        all_counts[col] = counts
    sys_prompt = f'Ты помощник аналитика. По сообщению пользователя, тебе необходимо определить значения зависимой переменной и группирующей. Исходи из того, что хочет пользователь.'\
                f'Вот данные, пользователя. dtypes: {data.dtypes.to_string()}\nvalues counts: {json.dumps(all_counts, ensure_ascii=False, indent=2)}'

    messages = state['messages']
    instruction = HumanMessage(content='Внимательно изучи последние системные сообщения об ошибках и исправь свой выбор переменных согласно им. Не повторяй предыдущие ошибки.')
    structured_llm = llm.with_structured_output(TargetExtraction)

    result = structured_llm.invoke([sys_prompt] + messages + [instruction])

    return {
        'target': result.target
        , 'grouping_variable': result.grouping_variable
        , 'grouping_values': result.grouping_values
    }

In [ ]:
class CorrectingVariables(BaseModel):
    answer: Literal['YES', 'NO'] = Field(description='Выбор удовлетворил пользователя? Толко YES или NO')
    correction_message: str = Field(description='Опиши переменные которые пользователь хочет поменять и на что, а какие хочет оставить. Если пользователя все устроило, оставь пробел.')

def is_correct_variables(state: UserState) -> UserState:
    print(f'Выбран таргет: {state['target']}, групповая переменная: {state['grouping_variable']}, ее значения: {' '.join(state['grouping_values'])}')
    response = input('Вас удовлетворяет выбор? ')
    structured_llm = llm.with_structured_output(CorrectingVariables)
    analysis = structured_llm.invoke(
        f'Пользователь ответил на выбор переменных. \n'
        f'Выбрано: target={state['target']}, group={state['grouping_variable']}. \n'
        f'Ответ пользователя: {response}'
    )
    new_messages = state.get('messages', [])
    if analysis.answer == 'NO':
        feedback_msg = SystemMessage(content=f"ОШИБКА: Пользователь недоволен. Исправь выбор. {analysis.correction_message}")
        new_messages.append(feedback_msg)
    return {
        'satisfied': analysis.answer,
        'messages': new_messages
    }

## другие функции

In [ ]:
def process_dtypes_strictly(state: UserState) -> UserState:
    data = state['data'].copy()
    t_col = state['target']
    g_col = state['grouping_variable']

    old_dtypes = data[[t_col, g_col]].dtypes
    selected_types = {}

    for col in [t_col, g_col]:
        unique_vals = set(data[col].dropna().unique())

        if unique_vals == {0, 1} or unique_vals == {0.0, 1.0}:
            new_type = 'string'
        else:
            new_type = str(data[col].dtype)

        selected_types[col] = new_type

        try:
            data[col] = data[col].astype(new_type)
        except Exception as e:
            print(f'Ошибка конвертации {col}: {e}')

    new_dtypes = data[[t_col, g_col]].dtypes

    if old_dtypes.to_string() != new_dtypes.to_string():
        artefact = (
            f'Изменены типы данных (бинарка 0/1 -> string):\n'
            f'БЫЛО:\n{old_dtypes.to_string()}\n'
            f'СТАЛО:\n{new_dtypes.to_string()}'
        )
    else:
        artefact = 'Типы данных без изменений.'

    return {
        'data': data,
        'target_dtype': selected_types[t_col],
        'grouping_variable_dtype': selected_types[g_col],
        'artefacts': state.get('artefacts', []) + [artefact]
    }

In [ ]:
def cleaning_target(state: UserState) -> UserState:
    data = state['data'].copy()
    target = state['target']
    grouping_variable = state['grouping_variable']

    lower_bound = None
    upper_bound = None

    if data[target].dtype in ['float64', 'int64']:
        q1 = data[target].quantile(0.25)
        q3 = data[target].quantile(0.75)
        iqr = q3 - q1
        lower_bound = q1 - 1.5 * iqr
        upper_bound = q3 + 1.5 * iqr

        data = data[(data[target] >= lower_bound) & (data[target] <= upper_bound)]

    if lower_bound is not None:
        count_removed = len(state['data']) - len(data)
        artefact = f'Были очищены выбросы: {count_removed}. Границы: {lower_bound:.2f} - {upper_bound:.2f}'
    else:
        artefact = f'Переменная {target} не является числовой, очистка выбросов пропущена.'

    if data[grouping_variable].dtype in ['float64', 'int64']:
        q1 = data[grouping_variable].quantile(0.25)
        q3 = data[grouping_variable].quantile(0.75)
        iqr = q3 - q1
        lower_bound = q1 - 1.5 * iqr
        upper_bound = q3 + 1.5 * iqr

        data = data[(data[grouping_variable] >= lower_bound) & (data[grouping_variable] <= upper_bound)]

    if lower_bound is not None:
        count_removed = len(state['data']) - len(data)
        artefact += f'Были очищены выбросы: {count_removed}. Границы: {lower_bound:.2f} - {upper_bound:.2f}'
    else:
        artefact += f'Переменная {grouping_variable} не является числовой, очистка выбросов пропущена.'

    return {
        'data': data,
        'artefacts': state.get('artefacts', []) + [artefact]
    }


# Узел перехода к стат тестам

In [ ]:
def identifying_variables(state: UserState) -> UserState:
    # target & grouping_variable
    data = state['data']
    groups = state['grouping_values']

    target = state['target']
    grouping_variable = state['grouping_variable']

    len_of_groups = len(groups)
    if len_of_groups == 1 or len_of_groups == 2:
        if is_numeric_dtype(data[target]) and is_numeric_dtype(data[grouping_variable]):
            if groups[0] == 'ALL':
                return 'ALL_NUMERIC' # две количественные
            else:
                return 'ONE_NUMERIC_AND_ONE_NOMINATIV' # одна номинативная друга количественная
        elif not is_numeric_dtype(data[target]) and not is_numeric_dtype(data[grouping_variable]):
            return 'ALL_NOMINATIV' # обе номинативные
        elif is_numeric_dtype(data[target]) and not is_numeric_dtype(data[grouping_variable]):
            return 'ONE_NUMERIC_AND_ONE_NOMINATIV' # одна номинативная друга количественная
    else:
        if is_numeric_dtype(data[target]):
            return 'MANY_GROUPS_AND_NUMERIC_TARGET' # таргет количественный
        elif not is_numeric_dtype(data[target]):
            return 'MANY_GROUPS_AND_NOMINATIV_TARGET' # таргет номинативный

# Обе переменные количественные

In [ ]:
# identifying_variables -> ALL_NUMERIC
import statsmodels.api as sm
from scipy.stats import pearsonr
from scipy.stats import spearmanr

def make_lin_reg(state: UserState) -> UserState:
    data = state['data']
    x = sm.add_constant(data[state['grouping_variable']])
    y = data[state['target']]
    model = sm.OLS(y, x).fit()
    artefact = f'По скольку в выбранных переменных обе переменные количественные, построим линейную регрессию. Сводка (summary):\n{str(model.summary())}'
    print('Уже строю линейную регрессию!')
    return {'artefacts': state.get('artefacts', []) + [artefact]}

def make_corrs(state: UserState) -> UserState:
    data = state['data']
    pr_stat, _ = pearsonr(data[state['target']], data[state['grouping_variable']])
    sp_stat, _ = spearmanr(data[state['target']], data[state['grouping_variable']])
    artefact = f'Построим корреляции между признаками, pearson: {pr_stat}, spearman: {sp_stat}'
    return {'artefacts': state.get('artefacts', []) + [artefact]}

# Обе переменнные номинативные

In [ ]:
# identifying_variables -> ALL_NOMINATIV
from scipy.stats import chi2_contingency
from scipy.stats import fisher_exact

def make_chi2(state: UserState) -> UserState:
    data = state['data']
    target = state['target']
    grouping_variable = state['grouping_variable']
    contingency_table = pd.crosstab(data[target], data[grouping_variable])
    res = chi2_contingency(contingency_table)
    print('Уже проверию Хи квадратом Пирсона!')
    artefact = f'Был проведен анализ таблицы сопряженности, вот таблица: {contingency_table}, вот результат (Хи-квадрат): {res}'
    return {'artefacts': state.get('artefacts', []) + [artefact]}

def make_fisher_exact(state: UserState) -> UserState:
    data = state['data']
    target = state['target']
    grouping_variable = state['grouping_variable']
    contingency_table = pd.crosstab(data[target], data[grouping_variable])
    res = fisher_exact(contingency_table)
    artefact = f'Также был проведен точный критерий фишера: {res}'
    return {'artefacts': state.get('artefacts', []) + [artefact]}

# Одна переменная количественная вторая номинативная

In [ ]:
# identifying_variables -> ONE_NUMERIC_AND_ONE_NOMINATIV
import matplotlib.pyplot as plt
import seaborn as sns
import io
import PIL.Image
from scipy.stats import probplot
from scipy.stats import shapiro
import google.generativeai as genai

genai.configure(api_key='')

class IsNormal(BaseModel):
    is_norm: bool = Field(description='Ответ True или False, распределение признака схоже с нормальным?')

vision_llm = genai.GenerativeModel(
    model_name='gemini-flash-latest',
    generation_config={
        'response_mime_type': 'application/json',
        'response_schema': IsNormal
    }
)

def is_normal_distribution(state: UserState) -> UserState:
    data = state['data']
    target = state['target']
    grouping_variable = state['grouping_variable']
    grouping_values = state['grouping_values']

    if not is_numeric_dtype(data[grouping_variable]):
        sample_one = data[data[grouping_variable] == grouping_values[0]][target].dropna().to_numpy()
        sample_two = data[data[grouping_variable] == grouping_values[1]][target].dropna().to_numpy()
    else:
        sample_one = data[data[grouping_variable] <= float(grouping_values[0])][target].dropna().to_numpy()
        sample_two = data[data[grouping_variable] > float(grouping_values[0])][target].dropna().to_numpy()

    samples = [sample_one, sample_two]
    group_names = [f"Группа {grouping_values[0]}", f"Группа {grouping_values[1]}"]

    results_is_normal = []
    total_artefacts = state.get('artefacts', [])

    for i, sample in enumerate(samples):
        n_obs = len(sample)
        stat, pval = shapiro(sample)

        if pval > 0.05:
            artefact = f'В группе {group_names[i]} признак {target} распределен нормально (Шапиро-Уилк, pval={pval:.4f})'
            total_artefacts.append(artefact)
            results_is_normal.append(True)
            continue

        fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(14, 6))
        probplot(sample, dist='norm', plot=axes[0])
        axes[0].set_title(f'QQ-plot: {group_names[i]}')
        sns.histplot(sample, bins=25, ax=axes[1], kde=True)
        axes[1].set_title(f'Распределение: {group_names[i]}')

        buf = io.BytesIO()
        fig.savefig(buf, format='png')
        buf.seek(0)
        img = PIL.Image.open(buf)

        prompt = (f'Распределение в группе "{group_names[i]}" схожее с нормальным? '
                  f'Размер выборки N={n_obs}. Тест Шапиро-Уилка: stat={stat:.4f}, pval={pval:.4f}. '
                  f'Учти ЦПМ: при больших N тест чувствителен. Если на графиках распределение близко к нормальному, '
                  f'можно пренебречь тестом.')

        response = vision_llm.generate_content([prompt, img])
        plt.close(fig)

        answer_is_normal = IsNormal.model_validate_json(response.text).is_norm
        results_is_normal.append(answer_is_normal)

        status_text = 'Нормально' if answer_is_normal else 'Ненормально'
        artefact = f'Визуальный анализ {group_names[i]}: {status_text}. (Шапиро-Уилк pval={pval:.4f})'
        total_artefacts.append(artefact)

    final_is_normal = all(results_is_normal)

    return {
        'is_normal_target': final_is_normal,
        'artefacts': total_artefacts
    }


/usr/local/lib/python3.12/dist-packages/google/colab/_import_hooks/_hook_injector.py:55: FutureWarning: 

All support for the `google.generativeai` package has ended. It will no longer be receiving 
updates or bug fixes. Please switch to the `google.genai` package as soon as possible.
See README for more details:

https://github.com/google-gemini/deprecated-generative-ai-python/blob/main/README.md

  loader.exec_module(module)


In [ ]:
def make_samples(state: UserState) -> UserState:
    data = state['data']
    target = state['target']
    grouping_variable = state['grouping_variable']
    grouping_values = state['grouping_values']
    if not is_numeric_dtype(data[grouping_variable]):
        sample_one = data[data[grouping_variable] == grouping_values[0]]
        sample_two = data[data[grouping_variable] == grouping_values[1]]
    else:
        sample_one = data[data[grouping_variable] <= float(grouping_values[0])]
        sample_two = data[data[grouping_variable] > float(grouping_values[1])]
    samples = [sample_one[target].to_numpy(), sample_two[target].to_numpy()]
    return {'samples': samples}

In [ ]:
from scipy.stats import levene
def choose_stat_test(state: UserState) -> UserState:
    sampl1, sampl2 = state['samples']
    is_normal_target = state['is_normal_target']

    if is_normal_target:
        stat, pval = levene(sampl1, sampl2)
        if pval > 0.05:
            return 'T_TEST'
        else:
            return 'WELCH_T_TEST'
    else:
        return 'MANAYITHUI' # охх обожаю, но остерегаюсь

In [ ]:
from scipy.stats import ttest_ind
def make_ttest(state: UserState) -> UserState:
    sampl1, sampl2 = state['samples']

    stat, pval = ttest_ind(sampl1, sampl2)

    artefact = f'Так как признак распределен нормально, и выборочные дисперсии равны (тест Левенне), мы проводим t-test, его результат: stat-{stat}, pval-{pval}.'

    return {'artefacts': state.get('artefacts', []) + [artefact]}

In [ ]:
def make_welch_ttest(state: UserState) -> UserState:
    sampl1, sampl2 = state['samples']

    stat, pval = ttest_ind(sampl1, sampl2, equal_var=False)

    artefact = f'Так как признак распределен нормально, и выборочные дисперсии не равны (тест Левенне), мы проводим t-test welch-а, его результат: stat-{stat}, pval-{pval}.'

    return {'artefacts': state.get('artefacts', []) + [artefact]}

In [ ]:
def make_manayithui(state: UserState):
    print('Применяю стат тест!')
    return

In [ ]:
import numpy as np
def choose_method_to_make_manayithui(state: UserState) -> UserState:
    sampl1, sampl2 = state['samples']
    bakets1 = np.array_split(sampl1, 500)
    bakets2 = np.array_split(sampl2, 500)

    if len(bakets1[0]) < 10 or len(bakets2[0]) < 10:
        return 'STANDART_MANAYITHUI'
    else:
        return 'BAKET_MANAYITHUI'

In [ ]:
from scipy.stats import mannwhitneyu
def make_standart_manayithui(state: UserState):
    sampl1, sampl2 = state['samples']
    stat, pval = mannwhitneyu(sampl1, sampl2)

    artefact = f'Так как признак распределен не нормально (Визуальный анализ и тест Шапиро Уилка) => используем стандартный стат тест Манна Уитни: stat-{stat}, pval-{pval}'

    return {'artefacts': state.get('artefacts', []) + [artefact]}

In [ ]:
from scipy.stats import mannwhitneyu
def make_baket_manayithui(state: UserState):
    sampl1, sampl2 = state['samples']

    s1_shuffled = np.random.permutation(sampl1)
    s2_shuffled = np.random.permutation(sampl2)

    bakets1 = np.array_split(s1_shuffled, 500)
    bakets2 = np.array_split(s2_shuffled, 500)

    baket1 = np.mean(bakets1, axis=1)
    baket2 = np.mean(bakets2, axis=1)

    stat, pval = mannwhitneyu(baket1, baket2)

    artefact = f'Так как признак распределен не нормально (Визуальный анализ и тест Шапиро Уилка). А также данных достаточно для бакетизации.'\
                f'Мы случайным образом будем создавать бакеты по >10 наблюдения в каждом, считать среднее и после проводить тест Манна Уитни.'\
                f'Что даст устойчивости и мощности тесту. \nstat-{stat}, pval-{pval}'

    return {'artefacts': state.get('artefacts', []) + [artefact]}

# Много групп и зависимая количественная

In [ ]:
def is_normal_distribution_for_var(state: UserState) -> UserState:
    data = state['data']
    target = state['target']
    grouping_variable = state['grouping_variable']

    groups = data[grouping_variable].unique()

    total_artefacts = state.get('artefacts', [])
    results_is_normal = []

    for group_label in groups:
        sample = data[data[grouping_variable] == group_label][target].dropna().to_numpy()

        if len(sample) < 3:
            results_is_normal.append(True)
            continue

        stat, pval = shapiro(sample)

        if pval > 0.05:
            artefact = f'Группа {group_label}: признак {target} распределен нормально (Шапиро-Уилк, pval={pval:.4f})'
            total_artefacts.append(artefact)
            results_is_normal.append(True)
            continue

        fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(14, 6))

        probplot(sample, dist='norm', plot=axes[0])
        axes[0].set_title(f'QQ-plot: Группа {group_label}')

        sns.histplot(sample, bins=20, ax=axes[1], kde=True)
        axes[1].set_title(f'Распределение: Группа {group_label}')

        buf = io.BytesIO()
        fig.savefig(buf, format='png')
        buf.seek(0)
        img = PIL.Image.open(buf)

        prompt = (f'Распределение группы "{group_label}" (признак {target}) схожее с нормальным? '
                  f'Размер выборки N={len(sample)}. Тест Шапиро-Уилка: stat={stat:.4f}, pval={pval:.4f}. '
                  f'Учти ЦПМ: при больших N тест чувствителен. Если на графиках распределение визуально нормальное, '
                  f'можешь подтвердить нормальность.')

        response = vision_llm.generate_content([prompt, img])
        plt.close(fig)

        answer_is_normal = IsNormal.model_validate_json(response.text).is_norm
        results_is_normal.append(answer_is_normal)

        status_text = 'Нормально' if answer_is_normal else 'Ненормально'
        artefact = f'Группа {group_label}: визуальный анализ показал "{status_text}". (Шапиро-Уилк pval={pval:.4f})'
        total_artefacts.append(artefact)

    final_is_normal = all(results_is_normal)

    return {
        'is_normal_target': final_is_normal,
        'artefacts': total_artefacts
    }

In [ ]:
def choose_stat(state: UserState) -> UserState:
    # is_normal_distribution -> data
    if state['is_normal_target']:
        data = state['data']
        target = state['target']
        grouping_variable = state['grouping_variable']

        if is_numeric_dtype(data[grouping_variable]):
            grouping_values = state['grouping_values']
            values = data['target']
            indices = np.searchsorted(np.sort(grouping_values), values)
            samples = [values[indices == i] for i in range(len(grouping_values) + 1)]
        else:
            samples = [group[target].values for name, group in data.groupby(grouping_variable)]

        samples = [i for i in samples if len(i) > 0]

        stat, pval = levene(*samples)

        if pval > 0.05:
            return {'method_of_var_analisis': 'ANOVA'}
    return {'method_of_var_analisis': 'KRUSKAL'}

In [ ]:
from scipy.stats import f_oneway
from scipy.stats import kruskal
def make_var_analisis(state: UserState) -> UserState:
    data = state['data']
    target = state['target']
    grouping_variable = state['grouping_variable']

    if is_numeric_dtype(data[grouping_variable]):
        grouping_values = state['grouping_values']
        values = data['target']
        indices = np.searchsorted(np.sort(grouping_values), values)
        samples = [values[indices == i] for i in range(len(grouping_values) + 1)]
    else:
        samples = [group[target].values for name, group in data.groupby(grouping_variable)]

    samples = [i for i in samples if len(i) > 0]
    if state['method_of_var_analisis'] == 'ANOVA':
        stat, pval = f_oneway(*samples)
    elif state['method_of_var_analisis'] == 'KRUSKAL':
        stat, pval = kruskal(*samples)

    artefact = f'Так как признак распределен {'нормально' if state['is_normal_target'] else 'ненормально'}, то был проведен стат тест - {state['method_of_var_analisis']}. Его результаты: stat-{stat}, pval-{pval}'
    return {'artefacts': state.get('artefacts', []) + [artefact]}

In [ ]:
def make_ling_var_regression(state: UserState) -> UserState:
    data = state['data'].copy()
    target = state['target']
    group_var = state['grouping_variable']
    group_vals = state['grouping_values']

    if not is_numeric_dtype(data[group_var]):
        data = data[data[group_var].isin(group_vals)]
        data = pd.get_dummies(data, columns=[group_var], drop_first=True).astype('float')
        features = [col for col in data.columns if col.startswith(f'{group_var}_')]
        x = sm.add_constant(data[features])
        y = data[target]
    else:
        y = (data[target] > group_vals[0]).astype(int)
        x = sm.add_constant(data[group_var])

    model = sm.OLS(y, x).fit()

    artefact = f'Была построенна линейная регрессия, вот ее сводка: {str(model.summary())}'
    return {'artefacts': state.get('artefacts', []) + [artefact]}

# Много групп и номинативный предиктор

In [ ]:
def make_logit_regression(state: UserState) -> UserState:
    data = state['data'].copy()
    target = state['target']
    group_var = state['grouping_variable']
    group_vals = state['grouping_values']

    if not is_numeric_dtype(data[group_var]):
        data = data[data[group_var].isin(group_vals)]
        x_raw = pd.get_dummies(data[group_var], drop_first=True).astype(float)
        x = sm.add_constant(x_raw)
        y = pd.get_dummies(data[target], drop_first=True).iloc[:, 0].astype(float)
    else:
        y = (data[target] > group_vals[0]).astype(int)
        x = sm.add_constant(data[group_var].astype(float))

    if len(data[target].unique()) == 2:
        print('Уже строю бинарную логистическую регрессию!')
        model = sm.Logit(y, x).fit()
    else:
        model = sm.MNLogit(y, x).fit()

    artefact = f'Была построенна логистическая регрессия, вот ее сводка: {str(model.summary())}'
    return {'artefacts': state.get('artefacts', []) + [artefact]}

# Сводка по артефактам

In [ ]:
def summary_by_llm(state: UserState) -> UserState:
    content = '\n'.join(state['artefacts'])
    #prompt = f'Ты помощник аналитика. Все исследование законченно, осталось только сделать КРАТКУЮ (100-200 слов) сводку. Также необходимо описать последованно этапы исследования с результатами.'\
    #'вот все артефакты исследования: {content}.'\
    #f'\n\nТак же можешь ознакомится с сообщениями во время исследования: {state['messages']}'

    prompt = f'Было проведено исследование. Теперь необходимо сделать коротку сводку, где требуется как бы рассказать все этапы анализа данных!\nВсе необходимые артефакты исследования здесь: {content}'\
    f'Для ознакомления, вот сообщения во время анализа данных: {state['messages']}'

    response = llm.invoke(prompt)
    return {'summary': response.content}

## инициализация графа

In [ ]:
graph = StateGraph(UserState)

In [ ]:
graph.add_node('find_variables', find_variables)

graph.add_node('is_correct_variables', is_correct_variables)

graph.add_node('correcting_dtypes', process_dtypes_strictly)

graph.add_node('cleaning_target', cleaning_target)

graph.add_node('make_lin_reg', make_lin_reg)
graph.add_node('make_corrs', make_corrs)

graph.add_node('make_chi2', make_chi2)
graph.add_node('make_fisher_exact', make_fisher_exact)

graph.add_node('is_normal_distribution', is_normal_distribution)

graph.add_node('make_samples', make_samples)

graph.add_node('make_ttest', make_ttest)
graph.add_node('make_welch_ttest', make_welch_ttest)
graph.add_node('make_manayithui', make_manayithui)
graph.add_node('make_standart_manayithui', make_standart_manayithui)
graph.add_node('make_baket_manayithui', make_baket_manayithui)

graph.add_node('choose_stat', choose_stat)
graph.add_node('is_normal_distribution_for_var', is_normal_distribution_for_var)
graph.add_node('make_var_analisis', make_var_analisis)
graph.add_node('make_ling_var_regression', make_ling_var_regression)

graph.add_node('make_logit_regression', make_logit_regression)

graph.add_node('summary_by_llm', summary_by_llm)

In [ ]:
graph.add_edge(START, 'find_variables')

graph.add_edge('find_variables', 'is_correct_variables')

graph.add_conditional_edges('is_correct_variables'
                            , lambda state: state['satisfied']
                            , {
                                'YES': 'correcting_dtypes'
                                , 'NO': 'find_variables'
                            })

graph.add_edge('correcting_dtypes', 'cleaning_target')

graph.add_conditional_edges('cleaning_target'
                            , identifying_variables
                            , {
                                'ALL_NUMERIC': 'make_lin_reg'
                                , 'ALL_NOMINATIV': 'make_chi2'
                                , 'ONE_NUMERIC_AND_ONE_NOMINATIV': 'is_normal_distribution'
                                , 'MANY_GROUPS_AND_NUMERIC_TARGET': 'is_normal_distribution_for_var'
                                , 'MANY_GROUPS_AND_NOMINATIV_TARGET': 'make_logit_regression'
                            })
graph.add_edge('make_lin_reg', 'make_corrs')
graph.add_edge('make_corrs', 'summary_by_llm')

graph.add_edge('make_chi2', 'make_fisher_exact')
graph.add_edge('make_fisher_exact', 'summary_by_llm')

graph.add_edge('is_normal_distribution', 'make_samples')

graph.add_conditional_edges('make_samples'
                            , choose_stat_test
                            , {
                                'T_TEST': 'make_ttest'
                                , 'WELCH_T_TEST': 'make_welch_ttest'
                                , 'MANAYITHUI': 'make_manayithui'
                            })
graph.add_edge('make_ttest', 'summary_by_llm')
graph.add_edge('make_welch_ttest', 'summary_by_llm')

graph.add_conditional_edges('make_manayithui'
                            , choose_method_to_make_manayithui
                            , {
                                'STANDART_MANAYITHUI': 'make_standart_manayithui'
                                , 'BAKET_MANAYITHUI': 'make_baket_manayithui'
                            })
graph.add_edge('make_standart_manayithui', 'summary_by_llm')
graph.add_edge('make_baket_manayithui', 'summary_by_llm')

graph.add_edge('is_normal_distribution_for_var', 'choose_stat')
graph.add_edge('choose_stat', 'make_var_analisis')
graph.add_edge('make_var_analisis', 'make_ling_var_regression')
graph.add_edge('make_ling_var_regression', 'summary_by_llm')

graph.add_edge('make_logit_regression', 'summary_by_llm')

In [ ]:
app = graph.compile()

In [ ]:
df = pd.DataFrame({
    'eat': ['Free'] * 1000 + ['Middle'] * 1000 + ['Lux'] * 450,
    'math_score': np.concatenate([
        np.random.normal(66, 34, 1000),
        np.random.normal(67, 34, 1000),
        np.random.normal(83, 15, 450),
    ])
})

In [ ]:
result = app.invoke({
    'data': df
    , 'messages': 'Привет! Если ученик питается платно, у него будет балл по математике выше?'
}, {'recursion_limit': 1000})

Выбран таргет: math_score, групповая переменная: eat, ее значения: Free Middle Lux
Вас удовлетворяет выбор? Давай только без люкса
Выбран таргет: math_score, групповая переменная: eat, ее значения: Free Middle
Вас удовлетворяет выбор? ага


In [ ]:
result.get('target')

'math_score'

In [ ]:
result.get('grouping_variable')

'eat'

In [ ]:
result.get('grouping_values')

['Free', 'Middle']

In [ ]:
result.get('messages')

[HumanMessage(content='Привет! Если ученик питается платно, у него будет балл по математике выше?', additional_kwargs={}, response_metadata={}, id='9ea903c0-55fc-4c45-b844-2b159e9db51c'),
 SystemMessage(content='ОШИБКА: Пользователь недоволен. Исправь выбор. Пользователь хочет оставить target=math_score и group=eat, но хочет изменить переменную люкса на без люкса.', additional_kwargs={}, response_metadata={}, id='19d7adee-0ffe-4dcf-95be-28ea902ee00f')]

In [ ]:
result.get('grouping_variable_dtype')

'object'

In [ ]:
result.get('target_dtype')

'float64'

In [ ]:
result.get('is_normal_target')

True

In [ ]:
result.get('summary')

'**Сводка исследования**\n\nЦелью исследования было проанализировать связь между типом питания учеников и их баллами по математике. Для этого было проведено исследование, в котором были проанализированы данные о баллах по математике и типе питания учеников.\n\n**Этапы анализа данных**\n\n1. **Очистка данных**: Были удалены выбросы из данных, что позволило получить более точные результаты. Границы для удаления выбросов были установлены в диапазоне от -9,64 до 150,93. В результате было удалено 32 выброса.\n2. **Визуальный анализ**: Были проведены визуальные анализы для двух групп: "Free" и "Middle". Результаты показали, что данные в обеих группах распределены нормально (тест Шапиро-Уилка: pval=0,0018 для группы "Free" и pval=0,0001 для группы "Middle").\n3. **Проверка равенства дисперсий**: Был проведен тест Левенне, который показал, что выборочные дисперсии равны. Это позволило использовать t-тест для сравнения средних баллов по математике между двумя группами.\n4. **t-тест**: Был прове

In [ ]:
print(result.get('summary'))

**Сводка исследования**

Целью исследования было проанализировать связь между типом питания учеников и их баллами по математике. Для этого было проведено исследование, в котором были проанализированы данные о баллах по математике и типе питания учеников.

**Этапы анализа данных**

1. **Очистка данных**: Были удалены выбросы из данных, что позволило получить более точные результаты. Границы для удаления выбросов были установлены в диапазоне от -9,64 до 150,93. В результате было удалено 32 выброса.
2. **Визуальный анализ**: Были проведены визуальные анализы для двух групп: "Free" и "Middle". Результаты показали, что данные в обеих группах распределены нормально (тест Шапиро-Уилка: pval=0,0018 для группы "Free" и pval=0,0001 для группы "Middle").
3. **Проверка равенства дисперсий**: Был проведен тест Левенне, который показал, что выборочные дисперсии равны. Это позволило использовать t-тест для сравнения средних баллов по математике между двумя группами.
4. **t-тест**: Был проведен t-тест

In [ ]:
def gen_png_graph(app_obj, name_photo: str = 'graph.png') -> None:
    with open(name_photo, 'wb') as f:
        f.write(app_obj.get_graph().draw_mermaid_png())

In [ ]:
gen_png_graph(app)